<a href="https://colab.research.google.com/github/SusimRoy/Real-Time-Mask-Detection/blob/main/Mask_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
pip install -q kaggle

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import files
files.upload()

In [8]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [9]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [11]:
!kaggle datasets download -d ashishjangra27/face-mask-12k-images-dataset

 95% 313M/330M [00:03<00:00, 100MB/s]
100% 330M/330M [00:03<00:00, 90.0MB/s]


In [ ]:
!unzip face-mask-12k-images-dataset.zip

In [13]:
import os
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

In [14]:
dir_type = ["Train","Test","Validation"]
cat = ["WithMask","WithoutMask"]

training_data = []
test_data = []
val_data=[]

IMG_SIZE = 150

path = "Face Mask Dataset/"

def create_data():
    for item in dir_type:
        data_path = path+item
        X = []
        y = []
        for folder in os.listdir(data_path):
            if cat.index(folder) == 0:
                labels = [1.0, 0.0]
            else:
                labels = [0.0, 1.0]
            for image in os.listdir(data_path+"/"+folder):
                img = cv2.imread(data_path+"/"+folder+"/"+image)
                img = cv2.resize(img, (125, 125))
                X.append(img)
                y.append(labels)
            if item == "Train":
                training_data.append((X, y))
            if item=="Test":
                test_data.append((X, y))
            if item=="Validation":
                val_data.append((X, y))

create_data()

In [15]:
train_images = np.array(training_data[0][0])/255.0
train_images = train_images.reshape(-1, 125, 125,3)
train_labels = np.array(training_data[0][1])

test_images = np.array(test_data[0][0])/255.0
test_images = test_images.reshape(-1, 125, 125, 3)
test_labels = np.array(test_data[0][1])

val_images = np.array(val_data[0][0])/255.0
val_images = val_images.reshape(-1, 125, 125, 3)
val_labels = np.array(val_data[0][1])
print(train_images.shape)
print(test_images.shape)
print(val_images.shape)

(10000, 125, 125, 3)
(992, 125, 125, 3)
(800, 125, 125, 3)


In [16]:
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Dense, Flatten
from tensorflow.keras.applications import VGG19


data_augmentation = keras.Sequential(
    [
        keras.layers.experimental.preprocessing.RandomFlip("horizontal"),
        keras.layers.experimental.preprocessing.RandomRotation(0.1),
        keras.layers.experimental.preprocessing.RandomZoom((0.2, 0.3)),
    ]
)



base_model = tf.keras.applications.VGG19(input_shape=(125, 125, 3), include_top=False, weights='imagenet')
for layer in base_model.layers[2:]:
    layer.trainable = False

flattening = tf.keras.layers.Flatten()
pre_layer1 = tf.keras.layers.Dense(256,activation="relu")
pre_layer2 = tf.keras.layers.Dense(64, activation="relu")
prediction_layer = tf.keras.layers.Dense(2, activation="softmax")

inputs = tf.keras.Input(shape=(125, 125, 3))
x = data_augmentation(inputs)
x = base_model(x, training=True)

x = flattening(x)
x = pre_layer1(x)
x=pre_layer2(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)
model.summary()

80142336/80134624 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 125, 125, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 125, 125, 3)       0         
_________________________________________________________________
vgg19 (Functional)           (None, 3, 3, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               1179904   
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
______________________________________________________________

In [17]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

epochs = 10
model.fit(train_images, train_labels, batch_size=32, epochs=epochs, validation_data=(val_images, val_labels), shuffle=True)

Epoch 1/10
313/313 [==============================] - 70s 114ms/step - loss: 0.0724 - accuracy: 0.9729 - val_loss: 0.0185 - val_accuracy: 0.9950
Epoch 2/10
313/313 [==============================] - 34s 110ms/step - loss: 0.0387 - accuracy: 0.9868 - val_loss: 0.0063 - val_accuracy: 0.9975
Epoch 3/10
313/313 [==============================] - 35s 112ms/step - loss: 0.0319 - accuracy: 0.9891 - val_loss: 0.0020 - val_accuracy: 1.0000
Epoch 4/10
313/313 [==============================] - 35s 113ms/step - loss: 0.0243 - accuracy: 0.9916 - val_loss: 0.0041 - val_accuracy: 1.0000
Epoch 5/10
313/313 [==============================] - 36s 114ms/step - loss: 0.0168 - accuracy: 0.9945 - val_loss: 0.0034 - val_accuracy: 0.9987
Epoch 6/10
313/313 [==============================] - 36s 115ms/step - loss: 0.0189 - accuracy: 0.9937 - val_loss: 0.0113 - val_accuracy: 0.9962
Epoch 7/10
313/313 [==============================] - 36s 116ms/step - loss: 0.0192 - accuracy: 0.9934 - val_loss: 0.0081 - val_ac

In [18]:
# score the model
score = model.evaluate(test_images, test_labels)
print("loss: ", score[0])
print("accuracy : ", score[1]*100, "%")

31/31 [==============================] - 2s 53ms/step - loss: 0.0147 - accuracy: 0.9990
loss:  0.014685255475342274
accuracy :  99.89919066429138 %


In [19]:
model.save("Mask_detect.h5")
model.save("./")

INFO:tensorflow:Assets written to: ./assets


In [21]:
new_model = tf.keras.models.load_model('Mask_detect.h5')

new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 125, 125, 3)]     0         
_________________________________________________________________
sequential (Sequential)      (None, 125, 125, 3)       0         
_________________________________________________________________
vgg19 (Functional)           (None, 3, 3, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               1179904   
_________________________________________________________________
dense_1 (Dense)              (None, 64)                16448     
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 130   

In [ ]:
#Model ready to use